In [1]:
import mediapipe

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize MediaPipe drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Initialize VideoCapture object
cap = cv2.VideoCapture(1)  # Change 0 to the camera index if you have multiple cameras

# Get the frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    results = pose.process(rgb_frame)

    # Create a black screen of the same size for each frame
    black_screen = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

    if results.pose_landmarks is not None:
        # Draw pose landmarks on the black screen
        mp_drawing.draw_landmarks(
            black_screen, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display the resulting frames
    cv2.imshow('Whole Body Detection', frame)
    cv2.imshow('Black Screen', black_screen)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [2]:
! pip install scipy

In [4]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import Delaunay

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize VideoCapture object
cap = cv2.VideoCapture(1)  # Change 0 to the camera index if you have multiple cameras

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    results = pose.process(rgb_frame)

    if results.pose_landmarks is not None:
        # Extract landmark coordinates
        landmarks = np.array([[lmk.x * frame.shape[1], lmk.y * frame.shape[0]] for lmk in results.pose_landmarks.landmark])

        # Perform Delaunay triangulation
        tri = Delaunay(landmarks)

        # Extract triangles
        triangles = landmarks[tri.simplices]

        # Draw triangles on frame
        for triangle in triangles:
            pt1, pt2, pt3 = triangle.astype(int)
            cv2.line(frame, tuple(pt1), tuple(pt2), (0, 255, 0), 1)
            cv2.line(frame, tuple(pt2), tuple(pt3), (0, 255, 0), 1)
            cv2.line(frame, tuple(pt3), tuple(pt1), (0, 255, 0), 1)

    # Display the resulting frame
    cv2.imshow('Advanced Body Scanning', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [13]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize VideoCapture object
cap = cv2.VideoCapture(1)  # Change 0 to the camera index if you have multiple cameras

# Get the frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the important landmarks for highlighting
important_landmarks = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

# Function to draw skeleton
def draw_skeleton(frame, landmarks):
    connections = mp_pose.POSE_CONNECTIONS
    for connection in connections:
        start_idx, end_idx = connection
        start_point = (int(landmarks[start_idx][0]), int(landmarks[start_idx][1]))
        end_point = (int(landmarks[end_idx][0]), int(landmarks[end_idx][1]))
        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)

# Function to draw highlighted landmarks on the black screen
def draw_highlighted_landmarks(frame, landmarks, important_landmarks):
    for idx, landmark in enumerate(landmarks):
        landmark_point = (int(landmark[0]), int(landmark[1]))
        if idx in important_landmarks:
            cv2.circle(frame, landmark_point, 4, (0, 0, 255), -1)  # Red color for important landmarks
        else:
            cv2.circle(frame, landmark_point, 2, (255, 255, 255), -1)  # White color for other landmarks

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Create a black screen of the same size for each frame
    black_screen = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    results = pose.process(rgb_frame)

    if results.pose_landmarks is not None:
        # Extract landmark coordinates
        landmarks = np.array([[lmk.x * frame.shape[1], lmk.y * frame.shape[0]] for lmk in results.pose_landmarks.landmark])

        # Draw skeleton on the black screen
        draw_skeleton(black_screen, landmarks)

        # Draw highlighted landmarks on the black screen
        draw_highlighted_landmarks(black_screen, landmarks, important_landmarks)

    # Display the black screen with highlighted landmarks
    cv2.imshow('Black Screen', black_screen)

    # Display the original frame with skeleton
    cv2.imshow('Whole Body Detection', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [41]:
! pip install screeninfo

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose and Hand models
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
pose = mp_pose.Pose()
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)

import screeninfo

# Get the resolution of the primary display
screen = screeninfo.get_monitors()[0]
screen_width = screen.width
screen_height = screen.height

# Initialize VideoCapture object with the display resolution
cap = cv2.VideoCapture(0)  # Change 0 to the camera index if you have multiple cameras
cap.set(cv2.CAP_PROP_FRAME_WIDTH, screen_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, screen_height)

# Define the important landmarks for highlighting
important_landmarks = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
                       32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
                       47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]

# Define finger landmark indices
finger_indices = [[0, 1, 2, 3, 4],   # Thumb
                  [0, 5, 6, 7, 8],   # Index finger
                  [0, 9, 10, 11, 12],  # Middle finger
                  [0, 13, 14, 15, 16],  # Ring finger
                  [0, 17, 18, 19, 20]]  # Little finger

# Function to draw skeleton
def draw_skeleton(frame, landmarks):
    connections = mp_pose.POSE_CONNECTIONS
    for connection in connections:
        start_idx, end_idx = connection
        start_point = (int(landmarks[start_idx][0]), int(landmarks[start_idx][1]))
        end_point = (int(landmarks[end_idx][0]), int(landmarks[end_idx][1]))
        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)

# Function to draw highlighted landmarks on the black screen
def draw_highlighted_landmarks(frame, landmarks, important_landmarks):
    for idx, landmark in enumerate(landmarks):
        landmark_point = (int(landmark[0]), int(landmark[1]))
        if idx in important_landmarks:
            cv2.circle(frame, landmark_point, 4, (0, 0, 255), -1)  # Red color for important landmarks
        else:
            cv2.circle(frame, landmark_point, 1, (255, 255, 255), -1)  # White color for other landmarks

# Function to draw fingers
def draw_fingers(frame, landmarks, finger_indices):
    for finger_index in finger_indices:
        for i in range(len(finger_index) - 1):
            start_point = (int(landmarks[finger_index[i]][0]), int(landmarks[finger_index[i]][1]))
            end_point = (int(landmarks[finger_index[i + 1]][0]), int(landmarks[finger_index[i + 1]][1]))
            cv2.line(frame, start_point, end_point, (255, 255, 255), 2)  # White color for finger lines

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Create a black screen of the same size for each frame
    black_screen = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose and Hand models
    pose_results = pose.process(rgb_frame)
    hand_results = hands.process(rgb_frame)

    # Draw pose landmarks on the black screen
    if pose_results.pose_landmarks is not None:
        pose_landmarks = np.array([[lmk.x * frame.shape[1], lmk.y * frame.shape[0]] for lmk in pose_results.pose_landmarks.landmark])
        draw_skeleton(black_screen, pose_landmarks)
        draw_highlighted_landmarks(black_screen, pose_landmarks, important_landmarks)

    # Draw hand landmarks and fingers on the black screen
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            hand_landmarks = np.array([[lmk.x * frame.shape[1], lmk.y * frame.shape[0]] for lmk in hand_landmarks.landmark])
            draw_highlighted_landmarks(black_screen, hand_landmarks, important_landmarks)
            draw_fingers(black_screen, hand_landmarks, finger_indices)

    # Display the black screen with highlighted landmarks
    cv2.imshow('Black Screen', black_screen)

    # Display the original frame with skeleton
    # cv2.imshow('Whole Body Detection', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
#handssssss

NameError: name 'frame_height' is not defined

In [36]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose, Hand, and Face models
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_detection
pose = mp_pose.Pose()
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_detection = mp_face.FaceDetection(min_detection_confidence=0.5)

# Initialize VideoCapture object
cap = cv2.VideoCapture(1)  # Change 0 to the camera index if you have multiple cameras

# Get the frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Convert POSE_CONNECTIONS to a list
POSE_CONNECTIONS = list(mp_pose.POSE_CONNECTIONS)

# Define the important landmarks for highlighting
important_landmarks = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
                       32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
                       47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]

# Define finger landmark indices
finger_indices = [[0, 1, 2, 3, 4],   # Thumb
                  [0, 5, 6, 7, 8],   # Index finger
                  [0, 9, 10, 11, 12],  # Middle finger
                  [0, 13, 14, 15, 16],  # Ring finger
                  [0, 17, 18, 19, 20]]  # Little finger

# Convert connections to a list
FACE_CONNECTIONS = [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10],
                    [10, 11], [11, 12], [12, 13], [13, 14], [14, 15], [15, 16], [16, 17], [17, 18], [18, 0], [19, 20],
                    [20, 21], [21, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30],
                    [30, 31], [31, 32], [32, 33], [33, 34], [34, 35], [35, 36], [36, 37], [37, 38], [38, 39], [39, 40],
                    [40, 41], [41, 42], [42, 19], [43, 44], [44, 45], [45, 46], [46, 47], [47, 48], [48, 49], [49, 50],
                    [50, 51], [51, 52], [52, 53], [53, 54], [54, 55], [55, 56], [56, 57], [57, 58], [58, 59], [59, 60],
                    [60, 61], [61, 62], [62, 63], [63, 64], [64, 43]]

# Function to draw skeleton
def draw_skeleton(frame, landmarks):
    connections = [(0, 1), (1, 14), (14, 16), (16, 18), (15, 17), (17, 19), (0, 2), (2, 3), (3, 4), (4, 6), (5, 7), (7, 9),
                   (6, 8), (8, 10), (11, 13), (12, 14), (23, 24), (11, 23), (12, 24), (11, 12)]
    for connection in connections:
        start_idx, end_idx = connection
        start_point = (int(landmarks[start_idx][0]), int(landmarks[start_idx][1]))
        end_point = (int(landmarks[end_idx][0]), int(landmarks[end_idx][1]))
        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)

# Function to draw highlighted landmarks on the black screen
def draw_highlighted_landmarks(frame, landmarks, important_landmarks):
    for idx, landmark in enumerate(landmarks):
        landmark_point = (int(landmark[0]), int(landmark[1]))
        if idx in important_landmarks:
            cv2.circle(frame, landmark_point, 4, (0, 0, 255), -1)  # Red color for important landmarks
        else:
            cv2.circle(frame, landmark_point, 1, (255, 255, 255), -1)  # White color for other landmarks

# Function to draw fingers
def draw_fingers(frame, landmarks, finger_indices):
    for finger_index in finger_indices:
        for i in range(len(finger_index) - 1):
            start_point = (int(landmarks[finger_index[i]][0]), int(landmarks[finger_index[i]][1]))
            end_point = (int(landmarks[finger_index[i + 1]][0]), int(landmarks[finger_index[i + 1]][1]))
            cv2.line(frame, start_point, end_point, (255, 255, 255), 2)  # White color for finger lines

# Function to draw face landmarks
def draw_face_landmarks(frame, face_landmarks):
    if len(face_landmarks) >= max([max(connection) for connection in FACE_CONNECTIONS]):
        for connection in FACE_CONNECTIONS:
            start_point = (int(face_landmarks[connection[0]][0]), int(face_landmarks[connection[0]][1]))
            end_point = (int(face_landmarks[connection[1]][0]), int(face_landmarks[connection[1]][1]))
            cv2.line(frame, start_point, end_point, (0, 255, 0), 2)


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Create a black screen of the same size for each frame
    black_screen = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose, Hand, and Face models
    pose_results = pose.process(rgb_frame)
    hand_results = hands.process(rgb_frame)
    face_results = face_detection.process(rgb_frame)

    # Draw pose landmarks on the black screen
    if pose_results.pose_landmarks is not None:
        pose_landmarks = np.array([[lmk.x * frame.shape[1], lmk.y * frame.shape[0]] for lmk in pose_results.pose_landmarks.landmark])
        draw_skeleton(black_screen, pose_landmarks)
        draw_highlighted_landmarks(black_screen, pose_landmarks, important_landmarks)

    # Draw hand landmarks and fingers on the black screen
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            hand_landmarks = np.array([[lmk.x * frame.shape[1], lmk.y * frame.shape[0]] for lmk in hand_landmarks.landmark])
            draw_highlighted_landmarks(black_screen, hand_landmarks, important_landmarks)
            draw_fingers(black_screen, hand_landmarks, finger_indices)

    # Draw face landmarks on the frame
    if face_results.detections:
        for detection in face_results.detections:
            face_landmarks = np.array([[lmk.x * frame_width, lmk.y * frame_height] for lmk in detection.location_data.relative_keypoints])
            draw_face_landmarks(frame, face_landmarks)

    # Display the black screen with highlighted landmarks
    cv2.imshow('Black Screen', black_screen)

    # Display the original frame with skeleton
    cv2.imshow('Whole Body Detection', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
